In [ ]:
import cv2
from imutils import face_utils
import pandas as pd
import numpy as np
import mediapipe as mp

import os
import re



In [12]:
import math

def average_mouth_deviation_percent(landmarks, image_width, image_height):
    def denormalize(landmark):
        return (landmark.x * image_width, landmark.y * image_height)

    # Уголки рта
    left = denormalize(landmarks[61])
    right = denormalize(landmarks[291])

    # Расстояние между уголками рта (основание линии)
    mouth_width = math.dist(left, right)

    # Прямая: Ax + By + C = 0
    A = right[1] - left[1]
    B = left[0] - right[0]
    C = right[0] * left[1] - left[0] * right[1]

    # Внутренний контур рта
    mouth_indices = [78, 80, 82, 13, 312, 310, 308, 14]

    deviations = []
    for idx in mouth_indices:
        x, y = denormalize(landmarks[idx])
        dist = abs(A * x + B * y + C) / math.sqrt(A**2 + B**2)
        deviations.append(dist)

    # Среднее отклонение в пикселях
    avg_dev_px = sum(deviations) / len(deviations)

    # Отклонение в процентах от ширины рта
    avg_dev_percent = (avg_dev_px / mouth_width) * 100 if mouth_width > 0 else 0

    return avg_dev_percent

In [13]:
DATASET_ALL_DATA = "../all_data/"

# Загружаем модель
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1)



In [14]:
def numeric_key(name):
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', name)]

In [15]:
def get_landmarks(image_path):
    image_array = np.fromfile(image_path, dtype=np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    if image is None:
        raise ValueError("Изображение не загружено!")
            
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    height, width = image_rgb.shape[:2]

    # Детектируем лицо
    faces = face_mesh.process(image_rgb)

    if not faces.multi_face_landmarks:
        return
        #raise ValueError("Лицо не найдено на изображении")

    shape = faces.multi_face_landmarks[0]


    return shape, height, width

In [16]:
def build_dataframe(dataset_dir, t_procents: list, f_procents: list, get_label_func = lambda k : k // 8): # получаем числовой код папки
    k = 0
    for label_dir in sorted(os.listdir(dataset_dir), key=numeric_key):
        label_path = os.path.join(dataset_dir, label_dir)
        if not os.path.isdir(label_path):
            continue
        
        # if get_label_func(k) % 2 == f:
        #     continue
        #label = get_label_func(k)
        #label = k % 8
        #label = 0 if label_dir == 'T' else 1  # T = 0, F = 1

        for filename in os.listdir(label_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(label_path, filename)

                result = get_landmarks(image_path)
                
                if result is None:
                    continue
                landmarks, h, w = result

                #if landmarks is None: continue
                
                avg_dev_percent = average_mouth_deviation_percent(
                    landmarks = landmarks.landmark,
                        image_width=w,
                        image_height=h)
                
                if get_label_func(k) % 2 == 0:
                    t_procents.append(avg_dev_percent)

                else:
                    f_procents.append(avg_dev_percent)
                
        k += 1


In [17]:
t_procents = []
f_procents = []

In [18]:
build_dataframe(DATASET_ALL_DATA, t_procents=t_procents, f_procents=f_procents)


In [19]:
print(len(t_procents))
print(len(f_procents))


37705
31426


In [20]:
print(f"Среднее отклонение для thinking: {sum(t_procents)/len(t_procents):.2f}% от длины рта")

print(f"Среднее отклонение для feeling: {sum(f_procents)/len(f_procents):.2f}% от длины рта")

Среднее отклонение для thinking: 3.60% от длины рта
Среднее отклонение для feeling: 3.49% от длины рта
